In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import requests
import pandas as pd
import re
from selenium.webdriver.support.ui import Select
from dataclasses import dataclass, field

In [4]:
df = pd.read_excel('cancasy_product_update.xlsx')

In [ ]:
replace_cat = {
    'x': '',
    '×': '',
}

for key, val in replace_cat.items():
    df[['ts_dimensions_length', 'ts_dimensions_width']] = df['size'].str.replace('سنتيمتر', '').str.split(, 1, expand=True)

In [12]:
df[['ts_dimensions_length', 'ts_dimensions_width']] = df['size'].str.replace('سنتيمتر', '').str.replace('x', '×').str.split('×', 1, expand=True)

In [10]:
df[['ts_dimensions_length', 'ts_dimensions_width']] = df['size'].str.replace('سنتيمتر', '').str.split('x', 1, expand=True)

In [13]:
df.to_excel('toto.xlsx')

In [ ]:
urls = [
   {'cat1': 'لوحات-ازهار', 'cat2': '', 'url': 'https://canvasy.net/product-category/%d9%84%d9%88%d8%ad%d8%a7%d8%aa-%d8%a7%d8%b2%d9%87%d8%a7%d8%b1/'},
]

In [4]:
from requests import request


url = 'https://midasfurniture.com/ar/home-furniture/living-rooms/sectional-sofa.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html.parser")

In [5]:
ttoo = soup.find_all('li', {'class': 'item product product-item'})
len(ttoo)

27

In [129]:
driver.get('https://canvasy.net/product-category/%d9%84%d9%88%d8%ad%d8%a7%d8%aa-%d8%a7%d8%b2%d9%87%d8%a7%d8%b1/page/9/')

In [ ]:
urls = []

In [71]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")
time.sleep(1)
products = soup.find_all('a', {'class': 'product-image-link'})
print(len(products))
liens = [toto['href']  for toto in products]
urls += liens

60


In [138]:
len(urls)

177

In [133]:
def scrap_product(driver, prozes=None, size=None):
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, "html.parser")
    name = soup.find('h1',{'class': 'product_title entry-title wd-entities-title'}).text.strip()
    try:
        prices = soup.find('div', {'class': 'single_variation_wrap'}).find_all('span', {'class': 'woocommerce-Price-amount amount'})
        try:
            price = prices[0].text.strip()
            special_price = prices[1].text.strip()
        except:
            price = prices[0].text.strip()
            special_price = ''
        
    except: 
        price =''
    description = soup.find('div', {'class': 'woocommerce-product-details__short-description'}).text.strip()
    cat1 = soup.find('a', {'class': 'breadcrumb-link breadcrumb-link-last'}).text.strip()
    sku = soup.find('span', {'class', 'sku'}).text.strip()
    try:
        qty = soup.find('p', {'class': 'stock in-stock'}).text.replace('متوفر في المخزون', '').strip()
    except:
        qty = ''
    images = soup.find_all('div', {'class': 'product-image-thumbnail'})
    len(images)
    list_images = [img.find('img')['src'].replace('-300x300', '') for img in images]
    base_image = list_images[0]
    add_images = ','.join(list_images[1:])
    data = {
        'sku': sku,
        'name': name,
        'price': price,
        'special_price': special_price,
        'link_url': driver.current_url,
        'qty': qty,
        'prozes': prozes,
        'size': size,
        'description': description,
        'base_image': base_image,
        'add_images': add_images,
        'cat1': cat1,
        'cat2': '',
        
    }
    return data


In [139]:
df = pd.read_excel('cancasy_product_model.xlsx')
def try_except(name):
        try:
            return name.first_selected_option.text
        except:
            return None
        
for i, url in enumerate(urls):
    print('Count: ', i)
    print('URL: ', url)
    driver.get(url)
    time.sleep(1)
    prozes = driver.find_elements_by_xpath('//table[@class="variations"]//ul[@class="variable-items-wrapper image-variable-wrapper"]//li')
    
    
    #select = Select(driver.find_element_by_id('pa_retsting'))
    count = len(prozes)
    for t in prozes:
        time.sleep(1)
        t.click()
        print('Dans le prozes')
        proz = t.find_element_by_tag_name('img').get_attribute('alt')
        print('prozes', t.find_element_by_tag_name('img').get_attribute('alt'))
        sizes1 = driver.find_elements_by_xpath('//table[@class="variations"]//select')
        sizes_id = sizes1[1].get_attribute('id')
        sizes = driver.find_elements_by_xpath(f'//select[@id="{sizes_id}"]//option')
        select = Select(driver.find_element_by_id(sizes_id))
        for i in range(1, len(sizes)):
            time.sleep(1)
            select.select_by_index(i)
            size =try_except(select)
            print('Size: ', size)
            data = scrap_product(driver, prozes=proz, size=size)
            df1 = pd.DataFrame([data])
            df = pd.concat([df, df1], ignore_index=True)
            df.to_excel('cancasy_product_update.xlsx')
            

            

Count:  0
URL:  https://canvasy.net/product/%d9%84%d9%88%d8%ad%d8%a7%d8%aa-%d8%ac%d8%af%d8%a7%d8%b1%d9%8a%d8%a9-%d9%85%d9%85%d9%8a%d8%b2%d8%a9-%d8%a8%d8%b9%d9%86%d9%88%d8%a7%d9%86-%d9%88%d8%b1%d8%af-%d8%a8%d8%b1%d8%aa%d9%82%d8%a7%d9%84%d9%8a/
Dans le prozes
prozes بدون برواز
Size:  100 × 120 سنتيمتر
Size:  120 × 140 سنتيمتر
Size:  140 × 200 سنتيمتر
Size:  40×60 سنتيمتر
Size:  60x90 سنتيمتر
Size:  80 × 120 سنتيمتر
Dans le prozes
prozes برواز اسود
Size:  100 × 120 سنتيمتر
Size:  120 × 140 سنتيمتر
Size:  140 × 200 سنتيمتر
Size:  40×60 سنتيمتر
Size:  60x90 سنتيمتر
Size:  80 × 120 سنتيمتر
Dans le prozes
prozes برواز خشبي
Size:  100 × 120 سنتيمتر
Size:  120 × 140 سنتيمتر
Size:  140 × 200 سنتيمتر
Size:  40×60 سنتيمتر
Size:  60x90 سنتيمتر
Size:  80 × 120 سنتيمتر
Dans le prozes
prozes برواز ذهبي
Size:  100 × 120 سنتيمتر
Size:  120 × 140 سنتيمتر
Size:  140 × 200 سنتيمتر
Size:  40×60 سنتيمتر
Size:  60x90 سنتيمتر
Size:  80 × 120 سنتيمتر
Dans le prozes
prozes برواز فضي
Size:  100 × 120 سنتيمتر
Size

IndexError: list index out of range

In [131]:
sizes1 = driver.find_elements_by_xpath('//table[@class="variations"]//select')
sizes_id = sizes1[1].get_attribute('id')
print(sizes_id)
sizes = driver.find_elements_by_xpath(f'//select[@id="{sizes_id}"]//option')
select = Select(driver.find_element_by_id(sizes_id))
for i in range(1, len(sizes)):
        try:
            time.sleep(1)
       
            size =try_except(select)
            select.select_by_index(i)
            print('Size: ', size)
            data = scrap_product(driver, prozes=proz, size=size)
            df1 = pd.DataFrame([data])
            df = pd.concat([df, df1], ignore_index=True)
            df.to_excel('cancasy_product_update.xlsx')
        except:
            continue

pa_retsting
Size:  80 × 80 سنتيمتر
Size:  100 × 100 سنتيمتر
Size:  120 × 120 سنتيمتر
Size:  40 × 40 سنتيمتر
Size:  60 × 60 سنتيمتر


In [70]:
prozes = driver.find_elements_by_xpath('//table[@class="variations"]//ul[@class="variable-items-wrapper image-variable-wrapper"]//li')
len(prozes)
for t in prozes:
    t.click()
    print('prozes', t.find_element_by_tag_name('img').get_attribute('alt'))
    time.sleep(2)


prozes بدون برواز
prozes برواز اسود
prozes برواز خشبي
prozes برواز ذهبي
prozes برواز فضي


In [112]:
toto = driver.find_elements_by_xpath('//table[@class="variations"]//select')
.get_attribute('id'):
        

pa_%d8%a7%d9%84%d8%a8%d8%b1%d9%88%d8%a7%d8%b2
pa_%d8%a7%d9%84%d9%85%d9%82%d8%a7%d8%b3


In [99]:
sizes = driver.find_elements_by_xpath('//select[@id="pa_retsting"]//option')
len(sizes)
select = Select(driver.find_element_by_id('pa_retsting'))
data = []
for i in range(1, len(sizes)):
    print('Count: ', i )
    select.select_by_index(i)
    size = try_except(select)
    print('Size: ', size)
    
    time.sleep(2)

Count:  1
Size:  100 × 120 سنتيمتر
Count:  2
Size:  120 × 140 سنتيمتر
Count:  3
Size:  140 × 200 سنتيمتر
Count:  4
Size:  40×60 سنتيمتر
Count:  5
Size:  60x90 سنتيمتر
Count:  6
Size:  80 × 120 سنتيمتر
